In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import scipy.stats as ss, scipy.integrate as integrate
import src.preprocess as pp

In [9]:
df_2021 = pd.read_csv('data/policecalls2021.csv')
df_2019 = pd.read_csv('data/policecalls2019.csv')

In [10]:
calltype_codes_with_text = df_2021[['CALLTYPE_CODE', 'CALL_TYPE']].drop_duplicates()
calltype_codes_with_text.to_csv('calltype_codes.csv')

In [11]:
calltype_codes_with_text

,CALLTYPE_CODE,CALL_TYPE
0,415FC,"DISTURBANCE, FIRECRACKERS"
3,602PC,TRESPASSING
11,1033A,"ALARM, AUDIBLE"
14,1057,FIREARMS DISCHARGED
17,1033S,"ALARM, SILENT"
...,...,...
826877,211AG,"ARMED ROBBERY, GANG RELATED"
1078671,300,W&I-UNDER JURIS OF JUV COURT
1105956,1095C,PEDESTRIAN STOP-CURFEW VIOL
1270839,601T,TRUANT (TABS)


In [12]:
dispo_codes_with_text = df_2021[['FINAL_DISPO_CODE', 'FINAL_DISPO']].drop_duplicates()
dispo_codes_with_text = dispo_codes_with_text.sort_values(by=['FINAL_DISPO_CODE'], axis=0)
dispo_codes_with_text.to_csv('dispo_codes.csv')

In [13]:
dispo_codes_with_text

,FINAL_DISPO_CODE,FINAL_DISPO
109,A,Arrest Made
1353,ADV,No Disposition
666,B,Arrest by Warrant
304,C,Criminal Citation
0,CAN,Canceled
109328,D,"Traffic Citation Issued, Hazardous Violation"
64,D,"Traffic Citation Issued, Hazar"
41,DUPNCAN,No Disposition
109408,E,"Traffic Citation Issued, Non-Hazardous Violation"
120,E,"Traffic Citation Issued, Non-H"


In [10]:
preprocessed_2021, standardized_2021, normalized_2021 = pp.pp_standardize_normalize(df_2021, 2021)
preprocessed_2019, standardized_2019, normalized_2019 = pp.pp_standardize_normalize(df_2019, 2019)

/Users/gabrielruiz/DSProjects/python/police-calls/src/preprocess.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_params['OFFENSE_DAY'] = (pd.to_datetime(df['OFFENSE_DATE']) + pd.to_timedelta(df['OFFENSE_TIME']) -
/Users/gabrielruiz/DSProjects/python/police-calls/src/preprocess.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_params['RESULT'] = df['FINAL_DISPO_CODE'].apply(


In [11]:
i = 0
names = ['preprocessed', 'standardized', 'normalized']
for gen_df in [preprocessed_2021, standardized_2021, normalized_2021]:
    gen_df.to_csv(f'data/preprocess/{names[i]}_2021.csv')
    i += 1

j = 0
for gen_df in [preprocessed_2019, standardized_2019, normalized_2019]:
    gen_df.to_csv(f'data/preprocess/{names[j]}_2019.csv')
    j += 1